# 2019 OK JPL Horizon vs OpenOrb comparison

## Assumptions/ accountability-sourced documentation:

* Values for chosen orbit found here: https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2019%20OK
* Location: L11, Gemini South
* Time: 2010/01/01 to 2020/01/01, midnights

## Current questions:

* Do I need to cite things in-code?
* Vertical or horizontal tables? Does it matter? (Because elements methods seem to be common) See below.
* How do I add units to the line that currently says 'object' (see cell 2) (may become irrelevant)
* Do we need a maximum difference function or class? What do I need to know about OOP for class things?
* Key error: 'R.A. (ICRF)' ??? what do ???
* "Global" variables for times- good or bad idea?
* oorb and python binding and in-line: what?
* Do my own thing or is borrowing from Siegfried a good idea in interest of time?

In [4]:
%matplotlib inline

import numpy as np
import astropy as ap
import astropy.units as u
from astropy.table import QTable
import matplotlib as plt
import astroquery as aq
from astroquery.jplhorizons import Horizons

In [5]:
# try Siegfried's thingamabob?


In [6]:
# example elements, trying to make one of start-date 2019OK
# id ok, time ok?, location isn't working. Locations tried: I11; I11@399
# no uri to check?? I am officially confused
# went back to example, and it's not using the time i specified. More confusion.
blastoff = '2010-01-01 00:00:00'
start = ap.time.Time(blastoff)
start1 = start.jd
print(start1)
obj = Horizons(id='433', location='500@10',
               epochs=start1)
print(obj.uri)
el = obj.elements()  
print(el)
print(obj.uri)

2455197.5
None
    targetname     datetime_jd ...         Q                 P        
       ---              d      ...         AU                d        
------------------ ----------- ... ----------------- -----------------
433 Eros (A898 PA)   2455197.5 ... 1.782955292957648 643.1050030280264
https://ssd.jpl.nasa.gov/horizons_batch.cgi?batch=1&TABLE_TYPE=ELEMENTS&MAKE_EPHEM=YES&OUT_UNITS=AU-D&COMMAND=%22433%3B%22&CENTER=%27500%4010%27&CSV_FORMAT=YES&ELEM_LABELS=YES&OBJ_DATA=YES&REF_SYSTEM=J2000&REF_PLANE=ECLIPTIC&TP_TYPE=ABSOLUTE&TLIST=2455197.5


In [7]:
#start_time = '2010-10-01 00:00:00'
#stop_time = '2020-10-01 00:00:00'

obj1 = Horizons(id='2019 OK', location='I11', epochs={'start': '2010-01-01 00:00:00', 
                                                    'stop': '2020-01-01 00:00:00', 
                                                    'step':'1d'})
obj2 = Horizons(id='2019 OK', location='I11', epochs= '2010-10-01 00:00:00')
#eph = obj1.ephemerides()
el = obj2.elements()
#print(eph)
print(el)

ValueError: Query failed without error message; check URI for more information

In [8]:
# Values table thing goes here
# do I even need this?

# vertical version

Element1 = ["e", "a", "q", "i", "node", "peri", "M", "t_p", "period", "n", "Q"]
Value1 = [.7571011351666629, 1.866845215104908, .4534545835685292, 2.089875053674596,
         302.0443761140395, 106.0972546563977, 101.8116827265041, 2458737.014932156260, 
         931.6673871166022, .3864039945781041, 3.280235846641287]
Unit1 = ["Null", "AU", "AU", "deg", "deg", "deg", "deg", "unsure", "days", "deg/d", "AU"]

hi = QTable([Element1, Value1, Unit1],
          names=('Element', 'Value', 'Units'),
          meta={'name': '2019OK data'})

# horizontal version

Element2 = ["e", "a", "q", "i", "node", "peri", "M", "t_p", "period", "n", "Q"]
Value2 = [.7571011351666629, 1.866845215104908, .4534545835685292, 2.089875053674596,
         302.0443761140395, 106.0972546563977, 101.8116827265041, 2458737.014932156260, 
         931.6673871166022, .3864039945781041, 3.280235846641287]
Value2 = np.array(Value2)
Value2 = np.transpose(Value2)
Unit2 = [u.dimensionless_unscaled, u.AU, u.AU, u.deg, u.deg, u.deg, u.deg, u.dimensionless_unscaled, 
         u.d, u.deg/u.d, u.AU]
bye = QTable(Value2*Unit2, names = Element2,)
print(hi)
bye

Element       Value        Units 
------- ------------------ ------
      e 0.7571011351666629   Null
      a  1.866845215104908     AU
      q 0.4534545835685292     AU
      i  2.089875053674596    deg
   node  302.0443761140395    deg
   peri  106.0972546563977    deg
      M  101.8116827265041    deg
    t_p  2458737.014932156 unsure
 period  931.6673871166022   days
      n 0.3864039945781041  deg/d
      Q  3.280235846641287     AU


e,a,q,i,node,peri,M,t_p,period,n,Q
object,object,object,object,object,object,object,object,object,object,object
0.7571011351666629,1.866845215104908 AU,0.4534545835685292 AU,2.089875053674596 deg,302.0443761140395 deg,106.0972546563977 deg,101.8116827265041 deg,2458737.014932156,931.6673871166022 d,0.3864039945781041 deg / d,3.280235846641287 AU


In [9]:
import pyoorb as oo
oo.pyoorb.oorb_init()

1

In [19]:
'''
This is where in-line things happen. 
Gfortran version 4.8.6

Useful bits from docs:
make ephem
oorb --task=propagation --epoch-mjd-utc=DATE --orb-in=ORBITFILEIN --orb-out=ORBITFILEOUT
oorb --task=ephemeris --orb-in=ORBBITFILEOUT
pyoorb.oorb_ephemeris_basic over pyoorb.oorb_ephemeris_full
Below is an example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation
It felt useful to study, I intend to shuffle things around to get to what I need from it
'''
orbits = np.array([[0, 1.46905, 0.33435, np.deg2rad(14.3024),
                    np.deg2rad(224.513), np.deg2rad(27.5419),
                    np.deg2rad(324.697), 3, 51544.5, 1, 12.5, 0.15]],
                    dtype=np.double, order='F')

epoch = np.array([51232.23, 3], dtype=np.double, order='F')
orb, err = oo.pyoorb.oorb_propagation(in_orbits=orbits,
                                     in_epoch=epoch,
                                    in_dynmodel='N')
                                    
mjds = np.arange(51232, 51233, 1/24)
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
eph, err = oo.pyoorb.oorb_ephemeris_full(in_orbits=orbits,
                                         in_obscode='568',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')
print(err, eph)

0 [[[ 5.12320000e+04  2.97420305e+01  9.39898382e+00  4.54411204e-01
    1.15967356e-01  2.55335063e+01  5.70687690e+01  1.92629817e+00
    2.27595666e+00  1.68784924e+01  7.56834978e+01  3.09658224e+01
   -2.57079287e+00 -1.22968568e+02 -2.57187230e+00  5.65262567e+01
   -3.03728791e+00 -9.74081340e+01 -3.03836734e+00  5.67373413e+01
    5.36581121e+01  2.79270091e+01  4.93916942e-01  3.23198414e+01
    1.06096650e+00  1.60454266e+00 -1.02066546e-01 -7.60331277e-03
    6.44843751e-03 -2.53113453e-03 -8.88643901e-01  4.34680947e-01
    1.86372479e-05  1.64854703e+02]
  [ 5.12320417e+04  2.97612057e+01  9.40381070e+00  4.53732760e-01
    1.15720650e-01  2.55263237e+01  5.70468341e+01  1.92635305e+00
    2.27643567e+00  1.68787982e+01  7.56922196e+01  3.09852619e+01
   -2.57280325e+00 -1.22990606e+02 -2.57364094e+00  5.65385408e+01
   -3.04034223e+00 -9.74373272e+01 -3.04117992e+00  7.00074489e+01
    4.35947788e+01  4.16228206e+01  4.97711138e-01  3.27693280e+01
    1.06064968e+00  1.60

# "Final" Draft Things Below This Line

In [68]:
# JPL integration, clean cell placeholder
# Values = .elements() something here?

In [69]:
# OpenOrb integration, clean cell placeholder

In [20]:
# difference function

def max_diff_metric(var1, var2):
    
    # don't forget to keep the 360deg problem in mind
    
    return RA_diff, DEC_diff